In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import torch.nn.functional as F
import numpy as np
import datetime
import warnings

# 경고 메시지 필터링
warnings.filterwarnings('ignore')

print("✅ 라이브러리 로드 완료")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# 데이터 파일 경로 설정 (로컬 경로)
DATA_DIR = "./"
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SUBMISSION_FILE = "sample_submission.csv"

# 전체 경로 생성
train_file_path = os.path.join(DATA_DIR, TRAIN_FILE)
test_file_path = os.path.join(DATA_DIR, TEST_FILE)
submission_file_path = os.path.join(DATA_DIR, SUBMISSION_FILE)

print(f"훈련 데이터 경로: {train_file_path}")
print(f"테스트 데이터 경로: {test_file_path}")
print(f"샘플 제출 파일 경로: {submission_file_path}")

# 파일 존재 여부 확인
required_files = [train_file_path, test_file_path, submission_file_path]
for file_path in required_files:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {file_path}")

print("✅ 모든 데이터 파일 확인 완료")


In [ ]:
# 데이터 로드
print("📊 데이터 로딩 중...")
train_data = pd.read_csv(train_file_path, encoding='utf-8')
test_data = pd.read_csv(test_file_path, encoding='utf-8')
submission_data = pd.read_csv(submission_file_path, encoding='utf-8')

# 데이터 정보 출력
print("📊 데이터셋 정보:")
print(f"Train 데이터: {train_data.shape}")
print(f"Test 데이터: {test_data.shape}")
print(f"Submission 데이터: {submission_data.shape}")

print("\n📝 Train 데이터 컬럼:", list(train_data.columns))
print("📝 Test 데이터 컬럼:", list(test_data.columns))

print("\n🎯 타겟 분포:")
print(train_data['generated'].value_counts())
print(f"\n클래스 비율 (Human:AI) = {train_data['generated'].value_counts()[0]}:{train_data['generated'].value_counts()[1]}")

# 데이터 기본 정보
print("\n📋 데이터 기본 정보:")
print(f"훈련 데이터 결측값: {train_data.isnull().sum().sum()}")
print(f"테스트 데이터 결측값: {test_data.isnull().sum().sum()}")


In [ ]:
# 모델 설정
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
NUM_CLASSES = 2
MAX_SEQUENCE_LENGTH = 512

print(f"🤖 사용할 모델: {MODEL_NAME}")
print(f"📏 최대 시퀀스 길이: {MAX_SEQUENCE_LENGTH}")

# 토크나이저 로드
print("🔧 토크나이저 로드 중...")
text_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("✅ 토크나이저 로드 완료")

# 분류 모델 로드
print("🔧 모델 로드 중...")
classification_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_CLASSES
)
print("✅ 모델 로드 완료")

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classification_model.to(device)
print(f"💻 사용 디바이스: {device}")

# 모델 파라미터 수 확인
total_params = sum(p.numel() for p in classification_model.parameters())
trainable_params = sum(p.numel() for p in classification_model.parameters() if p.requires_grad)
print(f"🔢 총 파라미터 수: {total_params:,}")
print(f"🔢 학습 가능한 파라미터 수: {trainable_params:,}")


In [ ]:
class CustomTextDataset(torch.utils.data.Dataset):
    """
    텍스트 분류를 위한 커스텀 데이터셋 클래스
    """
    def __init__(self, text_list, label_list=None, max_seq_len=512):
        print(f"📝 데이터셋 생성 중... (샘플 수: {len(text_list)})")
        self.text_encodings = text_tokenizer(
            text_list,
            truncation=True,
            padding=True,
            max_length=max_seq_len,
            return_tensors="pt"
        )
        self.target_labels = label_list
        print(f"✅ 토큰화 완료")

    def __getitem__(self, index):
        # 인코딩된 텍스트 데이터 추출
        data_item = {
            key: tensor[index] for key, tensor in self.text_encodings.items()
        }

        # 라벨이 있는 경우 추가
        if self.target_labels is not None:
            data_item["labels"] = torch.tensor(self.target_labels[index], dtype=torch.long)

        return data_item

    def __len__(self):
        return len(self.text_encodings["input_ids"])

print("✅ 커스텀 데이터셋 클래스 정의 완료")


In [ ]:
# 텍스트와 라벨 추출
print("📝 데이터 전처리 중...")
text_samples = train_data["full_text"].tolist()
label_samples = train_data["generated"].tolist()

print(f"총 샘플 수: {len(text_samples)}")
print(f"평균 텍스트 길이: {np.mean([len(text) for text in text_samples]):.1f}자")

# 텍스트 길이 분포 확인
text_lengths = [len(text) for text in text_samples]
print(f"텍스트 길이 분포:")
print(f"  - 최소: {min(text_lengths)}자")
print(f"  - 평균: {np.mean(text_lengths):.1f}자")
print(f"  - 최대: {max(text_lengths)}자")
print(f"  - 중간값: {np.median(text_lengths):.1f}자")

# 훈련/검증 데이터 분할 (95:5 비율)
X_train, X_validation, y_train, y_validation = train_test_split(
    text_samples,
    label_samples,
    test_size=0.05,
    stratify=label_samples,  # 클래스 비율 유지
    random_state=42
)

print(f"\n📊 데이터 분할 결과:")
print(f"훈련 데이터: {len(X_train)}개")
print(f"검증 데이터: {len(X_validation)}개")
print(f"훈련 데이터 클래스 분포: {np.bincount(y_train)}")
print(f"검증 데이터 클래스 분포: {np.bincount(y_validation)}")

# 데이터셋 객체 생성
print("\n🔧 데이터셋 객체 생성 중...")
print("🔄 훈련 데이터셋 생성...")
training_dataset = CustomTextDataset(X_train, y_train, MAX_SEQUENCE_LENGTH)

print("🔄 검증 데이터셋 생성...")
validation_dataset = CustomTextDataset(X_validation, y_validation, MAX_SEQUENCE_LENGTH)

print("🔄 테스트 데이터셋 생성...")
test_dataset = CustomTextDataset(test_data["paragraph_text"].tolist(), max_seq_len=MAX_SEQUENCE_LENGTH)

print("✅ 모든 데이터셋 생성 완료")


In [ ]:
# 훈련 파라미터 설정
EPOCHS = 10
BATCH_SIZE = 16  # 앨리스랩 환경에 맞게 조정
LEARNING_RATE = 2e-5
OUTPUT_DIR = "./results"
LOG_DIR = "./logs"

# 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

print(f"🔧 훈련 설정:")
print(f"  - 에포크: {EPOCHS}")
print(f"  - 배치 크기: {BATCH_SIZE}")
print(f"  - 학습률: {LEARNING_RATE}")
print(f"  - 출력 디렉토리: {OUTPUT_DIR}")
print(f"  - 로그 디렉토리: {LOG_DIR}")

# 예상 훈련 시간 계산
steps_per_epoch = len(X_train) // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS
print(f"  - 에포크당 스텝 수: {steps_per_epoch}")
print(f"  - 총 훈련 스텝 수: {total_steps}")

# TrainingArguments 설정
try:
    model_training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="roc-auc",
        greater_is_better=True,
        logging_dir=LOG_DIR,
        logging_steps=100,
        save_total_limit=3,
        report_to="none",
        seed=42,
        fp16=torch.cuda.is_available(),  # GPU 사용 시 mixed precision
        dataloader_num_workers=2,
        remove_unused_columns=False,
        warmup_steps=int(total_steps * 0.1),  # 10% 워밍업
        weight_decay=0.01
    )
    print("✅ 최신 버전 TrainingArguments 사용")
except TypeError:
    # 구버전 호환
    model_training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="roc-auc",
        greater_is_better=True,
        logging_dir=LOG_DIR,
        logging_steps=100,
        save_total_limit=3,
        report_to="none",
        seed=42
    )
    print("✅ 구버전 TrainingArguments 사용")


In [ ]:
def calculate_evaluation_metrics(evaluation_predictions):
    """
    모델 평가를 위한 메트릭 계산 함수
    """
    logits, true_labels = evaluation_predictions

    # 예측 클래스 계산
    predicted_classes = torch.argmax(torch.tensor(logits), dim=1).numpy()

    # 확률 계산 (클래스 1에 대한 확률)
    class_probabilities = F.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()

    # F1 점수 계산
    f1_score_value = f1_score(true_labels, predicted_classes)

    # 정확도 계산
    accuracy_value = (predicted_classes == true_labels).mean()

    # ROC-AUC 계산
    roc_auc_value = roc_auc_score(true_labels, class_probabilities)

    # 추가 메트릭 계산
    from sklearn.metrics import precision_score, recall_score
    precision_value = precision_score(true_labels, predicted_classes)
    recall_value = recall_score(true_labels, predicted_classes)

    return {
        "accuracy": accuracy_value,
        "f1": f1_score_value,
        "precision": precision_value,
        "recall": recall_value,
        "roc-auc": roc_auc_value
    }

print("✅ 평가 메트릭 함수 정의 완료")


In [ ]:
# Trainer 객체 생성
print("🔧 Trainer 설정 중...")
model_trainer = Trainer(
    model=classification_model,
    args=model_training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=calculate_evaluation_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("✅ Trainer 설정 완료")
print("🚀 모델 훈련 시작...")
print(f"예상 훈련 시간: 약 {(len(X_train) // BATCH_SIZE) * EPOCHS * 2 // 60}분")
print("=" * 50)

# 모델 훈련 실행
start_time = datetime.datetime.now()
training_results = model_trainer.train()
end_time = datetime.datetime.now()

training_duration = (end_time - start_time).total_seconds()
print("\n" + "=" * 50)
print("✅ 모델 훈련 완료!")
print(f"📈 최종 훈련 손실: {training_results.training_loss:.4f}")
print(f"⏱️ 훈련 소요 시간: {training_duration // 60:.0f}분 {training_duration % 60:.0f}초")
print(f"🔄 총 훈련 스텝: {training_results.global_step}")
print(f"⚡ 평균 속도: {training_results.global_step / training_duration:.2f} steps/sec")


In [ ]:
# 검증 데이터로 최종 평가
print("📊 검증 데이터로 최종 평가 중...")
eval_results = model_trainer.evaluate()
print("\n🎯 최종 검증 결과:")
print("=" * 40)
for key, value in eval_results.items():
    if key.startswith('eval_'):
        metric_name = key.replace('eval_', '').upper()
        print(f"  {metric_name}: {value:.4f}")
print("=" * 40)

# 테스트 데이터에 대한 예측 수행
print("\n🔮 테스트 데이터 예측 중...")
test_predictions = model_trainer.predict(test_dataset)

# 클래스 1에 대한 확률 계산
raw_probabilities = F.softmax(torch.tensor(test_predictions.predictions), dim=1)[:, 1].numpy()

print(f"📊 예측 완료 - {len(raw_probabilities)}개 샘플")
print(f"확률 통계:")
print(f"  - 최소값: {raw_probabilities.min():.4f}")
print(f"  - 최대값: {raw_probabilities.max():.4f}")
print(f"  - 평균값: {raw_probabilities.mean():.4f}")
print(f"  - 중간값: {np.median(raw_probabilities):.4f}")
print(f"  - 표준편차: {raw_probabilities.std():.4f}")

# 확률 분포 확인
print(f"\n확률 분포:")
print(f"  - 0.0-0.1: {((raw_probabilities >= 0.0) & (raw_probabilities < 0.1)).mean():.1%}")
print(f"  - 0.1-0.3: {((raw_probabilities >= 0.1) & (raw_probabilities < 0.3)).mean():.1%}")
print(f"  - 0.3-0.5: {((raw_probabilities >= 0.3) & (raw_probabilities < 0.5)).mean():.1%}")
print(f"  - 0.5-0.7: {((raw_probabilities >= 0.5) & (raw_probabilities < 0.7)).mean():.1%}")
print(f"  - 0.7-0.9: {((raw_probabilities >= 0.7) & (raw_probabilities < 0.9)).mean():.1%}")
print(f"  - 0.9-1.0: {((raw_probabilities >= 0.9) & (raw_probabilities <= 1.0)).mean():.1%}")

# 파워 튜닝 적용
POWER_TUNING_ALPHA = 1.1  # 파워 튜닝 계수
tuned_probabilities = np.clip(raw_probabilities ** POWER_TUNING_ALPHA, 0, 1)

print(f"\n⚡ 파워 튜닝 적용 (α={POWER_TUNING_ALPHA})")
print(f"튜닝 후 확률 통계:")
print(f"  - 최소값: {tuned_probabilities.min():.4f}")
print(f"  - 최대값: {tuned_probabilities.max():.4f}")
print(f"  - 평균값: {tuned_probabilities.mean():.4f}")
print(f"  - 중간값: {np.median(tuned_probabilities):.4f}")
print(f"  - 표준편차: {tuned_probabilities.std():.4f}")

# 제출 데이터에 예측 결과 할당
submission_data["generated"] = tuned_probabilities
print("✅ 예측 결과 할당 완료")


In [ ]:
# 제출 파일 저장
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
auc_score = eval_results.get('eval_roc-auc', 0)
submission_filename = f"koelectra_submission_{current_time}_auc_{auc_score:.4f}.csv"
final_submission_path = os.path.join("./", submission_filename)

# 제출 파일 저장
submission_data.to_csv(final_submission_path, index=False)

print("🎉 제출 파일 저장 완료!")
print(f"📁 파일 경로: {final_submission_path}")
print(f"📊 제출 데이터 크기: {submission_data.shape}")

# 예측 결과 요약 출력
print("\n📈 예측 결과 요약:")
print("=" * 30)
print(f"평균 확률: {tuned_probabilities.mean():.4f}")
print(f"표준편차: {tuned_probabilities.std():.4f}")
print(f"AI 생성 예측 비율:")
print(f"  - 확률 > 0.5: {(tuned_probabilities > 0.5).mean():.1%}")
print(f"  - 확률 > 0.3: {(tuned_probabilities > 0.3).mean():.1%}")
print(f"  - 확률 > 0.7: {(tuned_probabilities > 0.7).mean():.1%}")
print("=" * 30)

# 제출 파일 미리보기
print("\n👀 제출 파일 미리보기:")
print(submission_data.head(10))

# 모델 저장 (선택사항)
model_save_path = f"./koelectra_model_{current_time}"
print(f"\n💾 모델 저장 중: {model_save_path}")
model_trainer.save_model(model_save_path)
text_tokenizer.save_pretrained(model_save_path)
print(f"✅ 모델 저장 완료")


In [ ]:
# 실험 결과 요약
print("\n" + "="*60)
print("🎯 실험 결과 요약")
print("="*60)

# 기본 정보
print("📋 기본 정보:")
print(f"  - 모델: {MODEL_NAME}")
print(f"  - 실행 환경: 앨리스랩 클라우드")
print(f"  - 실행 시간: {current_time}")
print(f"  - 디바이스: {device}")

# 데이터 정보
print("\n📊 데이터 정보:")
print(f"  - 훈련 데이터 크기: {len(X_train):,}")
print(f"  - 검증 데이터 크기: {len(X_validation):,}")
print(f"  - 테스트 데이터 크기: {len(test_data):,}")
print(f"  - 클래스 분포 (Human:AI): {train_data['generated'].value_counts()[0]}:{train_data['generated'].value_counts()[1]}")

# 하이퍼파라미터
print("\n⚙️ 하이퍼파라미터:")
print(f"  - 배치 크기: {BATCH_SIZE}")
print(f"  - 학습률: {LEARNING_RATE}")
print(f"  - 최대 시퀀스 길이: {MAX_SEQUENCE_LENGTH}")
print(f"  - 에포크 수: {EPOCHS}")
print(f"  - 파워 튜닝 계수: {POWER_TUNING_ALPHA}")

# 훈련 결과
print("\n🏆 훈련 결과:")
print(f"  - 최종 훈련 손실: {training_results.training_loss:.4f}")
print(f"  - 훈련 시간: {training_duration // 60:.0f}분 {training_duration % 60:.0f}초")
print(f"  - 총 훈련 스텝: {training_results.global_step}")

# 검증 성능
print("\n📈 검증 성능:")
for key, value in eval_results.items():
    if key.startswith('eval_'):
        metric_name = key.replace('eval_', '').upper()
        print(f"  - {metric_name}: {value:.4f}")

# 제출 정보
print("\n📤 제출 정보:")
print(f"  - 제출 파일: {submission_filename}")
print(f"  - 모델 저장 위치: {model_save_path}")
print(f"  - 예측 평균 확률: {tuned_probabilities.mean():.4f}")

print("="*60)
print("✅ 실험 완료!")
print("="*60)
